In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pyiqa --quiet 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 104.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9

In [29]:
import traceback
import pyiqa
import os
from PIL import Image
import torch
import time
import pandas as pd
from pathlib import Path
import glob
import warnings
import csv

In [7]:
print("Loading NIQE model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
niqe_metric = pyiqa.create_metric('niqe', device=device)
print("model loaded")

2025-05-31 14:29:23,294 INFO: Network [NIQE] is created.
2025-05-31 14:29:23,295 INFO: Metric [NIQE] is created.


Loading NIQE model
Using device: cuda
model loaded


In [9]:
# create list to store data 
valid_images = [] 
valid_scores = [] 
corrupt_images = []
corrupted_errors = [] 

In [10]:
base_folder = "/kaggle/input/inaturalist-flower-images/test_images_200"
expected_species = ['Bellis_perennis', 'Leucanthemum_vulgare', 'Matricaria_chamomilla']

if os.path.exists(base_folder): 
    print(f"base folder found: {base_folder}")
    # list contents to verify structure 
    contents = os.listdir(base_folder)
    print(f"contents: {contents}")
    # check for seed folders 
    seed_folders = [item for item in contents if item.startswith('seed_')]
    print(f"Found seed folders: {seed_folders}")
else: 
    print(f"Base folder not found: {base_folder}")
    print("Please check the path!")

base folder found: /kaggle/input/inaturalist-flower-images/test_images_200
contents: ['experiment_summary.json', 'seed_123', 'seed_42', 'seed_456']
Found seed folders: ['seed_123', 'seed_42', 'seed_456']


In [16]:
seed_folders = [] 
base_path = Path(base_folder)
for folder in base_path.iterdir(): 
    if folder.is_dir() and folder.name.startswith('seed_'): 
        seed_folders.append(folder)
# sort folders for consistent processing order 
seed_folders.sort(key=lambda x: x.name)
print(f"Found {len(seed_folders)} seed folders:")
for folder in seed_folders: 
    print(f"  - {folder.name}")

Found 3 seed folders:
  - seed_123
  - seed_42
  - seed_456


In [21]:
all_species_folders = [] 
for seed_folder_path in seed_folders: 
    print(f"\nChecking species folders in: {seed_folder_path.name}")
    species_folders = []

    for species_name in expected_species: 
        species_path = seed_folder_path / species_name 
        if species_path.exists() and species_path.is_dir(): 
            species_folders.append(species_path)
            image_count = len([f for f in species_path.iterdir()
                              if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
            print(f"{species_name}: {image_count} images")
        else: 
            print(f" Warning: {species_name} folder not found in {seed_folder_path.name}")
    all_species_folders.extend(species_folders)
print(f"\nTotal species folders found: ")
    


Checking species folders in: seed_123
Bellis_perennis: 200 images
Leucanthemum_vulgare: 200 images
Matricaria_chamomilla: 200 images

Checking species folders in: seed_42
Bellis_perennis: 200 images
Leucanthemum_vulgare: 200 images
Matricaria_chamomilla: 200 images

Checking species folders in: seed_456
Bellis_perennis: 200 images
Leucanthemum_vulgare: 200 images
Matricaria_chamomilla: 200 images

Total species folders found: 


In [22]:
def process_single_image(image_path, niqe_metric): 
    start_time = time.perf_counter() 
    try: 
        img = Image.open(image_path).convert('RGB')
        img.verify() #Check image integrity 
        # Reload image after verification 
        img = Image.open(image_path).convert('RGB')
        # Calculate NIQE score 
        score = niqe_metric(img).item()
        end_time = time.perf_counter()
        processing_time = end_time - start_time
        return True, score, processing_time
    except Exception as e: 
        end_time = time.perf_counter()
        processing_time = end_time - start_time
        return False, str(e), processing_time
print("Image processing function defined")

Image processing function defined


In [34]:
# Process each seed folder and create separate csv files 
seed_number = int(input('Enter seed number (42, 123, 456): '))
seed_name = f"seed_{seed_number}"  # Convert to proper folder name
seed_folder_path = base_path / seed_name
print(f"Processing {seed_name}")

# Storage for this seed's results
seed_valid_results = [] 
seed_corrupted_results = [] 

# Get species folders for this seed 
seed_species_folders = [] 
for species_name in expected_species:  # Fix: use EXPECTED_SPECIES (uppercase)
    species_path = seed_folder_path / species_name 
    if species_path.exists() and species_path.is_dir(): 
        seed_species_folders.append(species_path)

# Process each species in seed
start_time = time.perf_counter()

for species_folder_path in seed_species_folders:  # Fix: use seed_species_folders instead of species_folders
    species_name = species_folder_path.name
    print(f"Processing {species_name}...")
    
    # Get all image files 
    image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
    image_files = [] 
    for ext in image_extensions: 
        image_files.extend(list(species_folder_path.glob(f"*{ext}")))
    
    print(f"Found {len(image_files)} images")
    
    # Process each image 
    for i, image_path in enumerate(image_files, 1): 
        if i % 50 == 0 or i == 1: 
            print(f"[{i}/{len(image_files)}] Processing...")
        
        success, result, proc_time = process_single_image(image_path, niqe_metric)
        
        if success:
            seed_valid_results.append({  # Fix: use seed_valid_results
                'Seed_Folder': seed_name,
                'Species': species_name,
                'Image_Path': str(image_path),
                'Image_Name': image_path.name,
                'NIQE_Score': result,
                'Processing_Time_Seconds': proc_time  # Add processing time back
            })
        else:
            seed_corrupted_results.append({  # Fix: use seed_corrupted_results
                'Seed_Folder': seed_name,
                'Species': species_name,
                'Image_Path': str(image_path),
                'Image_Name': image_path.name,
                'Error': result,
                'Processing_Time_Seconds': proc_time  # Add processing time back
            })

output_csv = f"niqe_resutls_{seed_number}.csv"
print(f"saving resutls to {output_csv}")

with open(output_csv, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['seed_folder', 'species', 'image_path', 'image_name', 'niqe_score', 'processing_time'])
        # Write each result as a separate row
    for result in seed_valid_results:
        writer.writerow([
            result['Seed_Folder'],
            result['Species'], 
            result['Image_Path'],
            result['Image_Name'],
            result['NIQE_Score'],
            result['Processing_Time_Seconds']
        ])
print('Results saved successfully')

from IPython.display import FileLink, display
print(f"\nDownload results: ")
display(FileLink(output_csv))

Enter seed number (42, 123, 456):  456


Processing seed_456
Processing Bellis_perennis...
Found 200 images
[1/200] Processing...
[50/200] Processing...
[100/200] Processing...
[150/200] Processing...
[200/200] Processing...
Processing Leucanthemum_vulgare...
Found 200 images
[1/200] Processing...
[50/200] Processing...
[100/200] Processing...
[150/200] Processing...
[200/200] Processing...
Processing Matricaria_chamomilla...
Found 200 images
[1/200] Processing...
[50/200] Processing...
[100/200] Processing...
[150/200] Processing...
[200/200] Processing...
saving resutls to niqe_resutls_456.csv
Results saved successfully

Download results: 


/kaggle/working/niqe_resutls_456.csv